# Imports

In [1]:
import nbimporter
from my_functions import nested_cross_val

#Python3
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

Importing Jupyter notebook from my_functions.ipynb


In [2]:
import xgboost as xgb

#### DATA

In [3]:
df = pd.read_csv("../../../data/datasets/regression_final(domestic).csv", index_col=0)
df=df.drop(columns=['profile','reviews'])

In [4]:
from pprint import pprint

clf = xgb.XGBRegressor()
print(f"Parameters currently in use:")
pprint(clf.get_params())

Parameters currently in use:
{'base_score': None,
 'booster': None,
 'colsample_bylevel': None,
 'colsample_bynode': None,
 'colsample_bytree': None,
 'gamma': None,
 'gpu_id': None,
 'importance_type': 'gain',
 'interaction_constraints': None,
 'learning_rate': None,
 'max_delta_step': None,
 'max_depth': None,
 'min_child_weight': None,
 'missing': nan,
 'monotone_constraints': None,
 'n_estimators': 100,
 'n_jobs': None,
 'num_parallel_tree': None,
 'objective': 'reg:squarederror',
 'random_state': None,
 'reg_alpha': None,
 'reg_lambda': None,
 'scale_pos_weight': None,
 'subsample': None,
 'tree_method': None,
 'validate_parameters': None,
 'verbosity': None}


### Select Random Grid

In [5]:
max_depth= [0]+[i for i in range(1,50,5)]
min_child_weight= [i for i in range(1, 21, 2)]
n_estimators= [i for i in range(200,2000,200)]
learning_rate= [0.03, 0.05, 0.1,0.15]
gamma=[0,1,2,3,4,5]
max_leaves = [0,31]+[i for i in range(100,1000,100)]
max_bin = [255]+[i for i in range(510, 510*5, 510)]
random_grid = {'max_depth': max_depth,
               'min_child_weight': min_child_weight,
               'n_estimators': n_estimators,
              'learning_rate': learning_rate,
               'gamma': gamma,
               'max_leaves': max_leaves,
               'max_bin': max_bin
              }
pprint(random_grid)

{'gamma': [0, 1, 2, 3, 4, 5],
 'learning_rate': [0.03, 0.05, 0.1, 0.15],
 'max_bin': [255, 510, 1020, 1530, 2040],
 'max_depth': [0, 1, 6, 11, 16, 21, 26, 31, 36, 41, 46],
 'max_leaves': [0, 31, 100, 200, 300, 400, 500, 600, 700, 800, 900],
 'min_child_weight': [1, 3, 5, 7, 9, 11, 13, 15, 17, 19],
 'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800]}


### Nested Random Grid Search

In [6]:
%%time
seed=42
model = xgb.XGBRegressor
rs_eval = nested_cross_val(
  df=df,
  model=model,
  space=random_grid,
  n_iter=200,
  scoring='neg_mean_absolute_error',
  search_cv='random',
  outer=10,
  inner=3,
  seed=42
)

>$USD=19043860.814, mape=26.391 mae=19043860.814,est=-0.279, cfg={'n_estimators': 1000, 'min_child_weight': 13, 'max_leaves': 700, 'max_depth': 6, 'max_bin': 1020, 'learning_rate': 0.03, 'gamma': 0}
>$USD=11385130.544, mape=38.005 mae=11385130.544,est=-0.290, cfg={'n_estimators': 1000, 'min_child_weight': 13, 'max_leaves': 700, 'max_depth': 6, 'max_bin': 1020, 'learning_rate': 0.03, 'gamma': 0}
>$USD=7678323.453, mape=367.076 mae=7678323.453,est=-0.288, cfg={'n_estimators': 1000, 'min_child_weight': 13, 'max_leaves': 700, 'max_depth': 6, 'max_bin': 1020, 'learning_rate': 0.03, 'gamma': 0}
>$USD=6028545.047, mape=56.251 mae=6028545.047,est=-0.290, cfg={'n_estimators': 1000, 'min_child_weight': 13, 'max_leaves': 700, 'max_depth': 6, 'max_bin': 1020, 'learning_rate': 0.03, 'gamma': 0}
>$USD=10472620.928, mape=151.394 mae=10472620.928,est=-0.305, cfg={'n_estimators': 1800, 'min_child_weight': 7, 'max_leaves': 300, 'max_depth': 21, 'max_bin': 2040, 'learning_rate': 0.05, 'gamma': 0}
>$USD=8

### Evaluate Results

In [ ]:
base_performance(df,model,outer=10,seed=seed,tuned_mae=tuned_mae)

### Fine Tune Parameter Grid Search

In [ ]:
tuned_grid = {
  'num_leaves': [300,400,500], 
  'max_bin': [175,255,510],
  'num_iterations': [700,800,900], 
  'learning_rate':[0.05,0.1,0.15],
  'boosting_type': ['gbdt', 'dart'],
  'max_depth': [-1, 5, 10]
}

In [ ]:
grid_nested_cross_val(df=df,outer=10,inner=3,model=model,space=tuned_grid, n_iter=200,scoring='neg_mean_absolute_error',seed=seed)

In [7]:
'''
  REPORT RESULTS
'''
rs_eval.to_csv("xgb_results.csv")